# Etapa 4 - Modelagem com Machine Learning

## Bibliotecas e Carregamento do Parquet Filtrado

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_parquet("filtradoParquet.parquet")
print(df.shape)
df.head()


## Feature Engineering


In [ ]:
# Mapeamento da região 
map_regiao = {'CO': 1, 'N': 2, 'NE': 3, 'S': 4, 'SE': 5}
df['REGIAO'] = df['REGIAO'].map(map_regiao).fillna(df['REGIAO'])

# Garantir que MES e IDADE_FAIXA estejam corretos
df["DTOBITO"] = df["DTOBITO"].astype(str)
df["MES"] = df["DTOBITO"].str[:-4].str[-2:].str.zfill(2).astype(int)
df["IDADE_FAIXA"] = (df["IDADE"] // 10) * 10

# Converter todas as colunas possíveis para numérico
for col in df.columns:
    if col not in ["DTOBITO", "NECROPSIA", "TPPOS"]:
        df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0).astype(int)

# Criar variável indicadora de óbito por doença respiratória
df["ALVO"] = (df["DOENCA_RESPIRATORIA"] == 1).astype(int)

# Renomeando coluna de escolaridade
df.rename(columns={'ESC2010': 'ESCOLARIDADE'}, inplace=True)

print(df.dtypes)


In [ ]:
# Agrupar por Região, Escolaridade e Mês
grupo = df.groupby(["REGIAO", "ESCOLARIDADE", "MES"])

# Total de óbitos respiratórios por grupo no mês
agregado = grupo["ALVO"].sum().reset_index(name="obitos_mes")

# Função auxiliar para calcular agregações temporais coerentes
def add_temporal_features(df_agg):
    df_agg = df_agg.sort_values(["REGIAO", "ESCOLARIDADE", "MES"])
    df_agg["media_3m_obitos"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["obitos_mes"].transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
    df_agg["variacao_obitos_1m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["obitos_mes"].transform(lambda x: (x - x.shift(1)) / x.shift(1))
    df_agg["desvio_padrao_3m_obitos"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["obitos_mes"].transform(lambda x: x.shift(1).rolling(3, min_periods=1).std())

    # Média e desvio de idade_faixa
    idade = grupo["IDADE_FAIXA"].mean().reset_index(name="media_idade_faixa")
    df_agg = df_agg.merge(idade, on=["REGIAO", "ESCOLARIDADE", "MES"], how="left")

    # Moda de raça/cor
    racacor = grupo["RACACOR"].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan).reset_index(name="moda_racacor")
    df_agg = df_agg.merge(racacor, on=["REGIAO", "ESCOLARIDADE", "MES"], how="left")

    # Proporção de assistência médica
    assistmed = grupo["ASSISTMED"].mean().reset_index(name="prop_assistmed_sim")
    df_agg = df_agg.merge(assistmed, on=["REGIAO", "ESCOLARIDADE", "MES"], how="left")

    df_agg["variacao_assistmed_1m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["prop_assistmed_sim"].transform(lambda x: x - x.shift(1))

    # Proporção de óbitos em hospital (LOCOCOR=1)
    lococor = grupo.apply(lambda g: (g["LOCOCOR"] == 1).mean()).reset_index(name="prop_lococor_hospital")
    df_agg = df_agg.merge(lococor, on=["REGIAO", "ESCOLARIDADE", "MES"], how="left")

    df_agg["variacao_lococor_hospital_3m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["prop_lococor_hospital"].transform(lambda x: x - x.shift(1).rolling(3, min_periods=1).mean())

    # Média e desvio da idade (últimos 3 meses)
    df_agg["media_idade_3m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["media_idade_faixa"].transform(lambda x: x.shift(1).rolling(3, min_periods=1).mean())
    df_agg["desvio_idade_3m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["media_idade_faixa"].transform(lambda x: x.shift(1).rolling(3, min_periods=1).std())

    # Proporção da região no total nacional no mês
    total_mes = df_agg.groupby("MES")["obitos_mes"].transform("sum")
    df_agg["prop_regiao_ativa"] = df_agg["obitos_mes"] / total_mes

    # Tendência linear (coeficiente) dos últimos 3 meses
    def trend(x):
        if len(x) < 3: return 0
        t = np.arange(len(x))
        return np.polyfit(t, x, 1)[0]
    df_agg["tendencia_3m"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["obitos_mes"].transform(lambda x: x.shift(1).rolling(3, min_periods=3).apply(trend, raw=False))

    # Crescimento acumulado nos últimos 3 meses
    df_agg["crescimento_acumulado"] = df_agg.groupby(["REGIAO", "ESCOLARIDADE"])["variacao_obitos_1m"].transform(lambda x: x.shift(1).rolling(3, min_periods=1).sum())

    return df_agg

agregado = add_temporal_features(agregado)

print("Formato final das features:", agregado.shape)
agregado.head(100)

## Evolução temporal das mortes respiratórias por escolaridade e região 

In [ ]:
# Coluna binária indicando óbito por doença respiratória
col_mortes = 'DOENCA_RESPIRATORIA'

# Evolução por Escolaridade
mortes_escolaridade = (
    df.groupby(['MES', 'ESCOLARIDADE'])[col_mortes]
    .sum()
    .reset_index()
    .sort_values(['ESCOLARIDADE', 'MES'])
)

plt.figure(figsize=(8,5))
for esc in mortes_escolaridade['ESCOLARIDADE'].unique():
    subset = mortes_escolaridade[mortes_escolaridade['ESCOLARIDADE'] == esc]
    plt.plot(subset['MES'], subset[col_mortes], marker='o', label=f'Escolaridade {esc}')
plt.title("Evolução das mortes respiratórias por Escolaridade (Jan–Nov)")
plt.xlabel("Mês")
plt.ylabel("Número de Óbitos Respiratórios")
plt.legend()
plt.grid(True)
plt.show()

# Evolução por Região 
mortes_regiao = (
    df.groupby(['MES', 'REGIAO'])[col_mortes]
    .sum()
    .reset_index()
    .sort_values(['REGIAO', 'MES'])
)

plt.figure(figsize=(8,5))
for reg in mortes_regiao['REGIAO'].unique():
    subset = mortes_regiao[mortes_regiao['REGIAO'] == reg]
    plt.plot(subset['MES'], subset[col_mortes], marker='o', label=f'Região {reg}')
plt.title("Evolução das mortes respiratórias por Região (Jan–Nov)")
plt.xlabel("Mês")
plt.ylabel("Número de Óbitos Respiratórios")
plt.legend()
plt.grid(True)
plt.show()


## Preparação dos Dados

In [ ]:
# Definir features e alvo
features = [
    'media_3m_obitos', 'variacao_obitos_1m', 'desvio_padrao_3m_obitos',
    'media_idade_faixa', 'moda_racacor', 'prop_assistmed_sim',
    'variacao_assistmed_1m', 'prop_lococor_hospital', 'variacao_lococor_hospital_3m',
    'media_idade_3m', 'desvio_idade_3m', 'prop_regiao_ativa',
    'tendencia_3m', 'crescimento_acumulado', 'ESCOLARIDADE'
]

target = 'obitos_mes'

# Separar treino (meses 1–11) e teste (mês 12)
X_train = agregado[agregado["MES"] <= 11][features]
y_train = agregado[agregado["MES"] <= 11][target]
X_test  = agregado[agregado["MES"] == 12][features]
y_test  = agregado[agregado["MES"] == 12][target]

# Substitui infinitos e -inf por NaN
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Substitui NaN por zero
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

print("Treino:", X_train.shape, "| Teste:", X_test.shape)

## Modelagem - Regressão Linear



In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

# MAE, RMSE, R²
mae_lr = mean_absolute_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mean_squared_error(y_test, y_pred_lr))
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression")
print(f"  MAE  : {mae_lr:.2f} óbitos")
print(f"  RMSE : {rmse_lr:.2f} óbitos")
print(f"  R²   : {r2_lr:.3f}")


## Modelagem - Random Forest

In [ ]:
rf = RandomForestRegressor(
    n_estimators=7000, # número de árvores
    max_depth=16, # profundidade máxima de cada árvore
    random_state=42 # semente para o gerador de números aleatórios
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)

print("Random Forest")
print(f"  MAE  : {mae_rf:.2f} óbitos")
print(f"  RMSE : {rmse_rf:.2f} óbitos")
print(f"  R²   : {r2_rf:.3f}")


## Modelagem - XGBoost

In [ ]:
xgb = XGBRegressor(
    n_estimators=100, # número de árvores
    learning_rate=0.10, # taxa de aprendizado
    max_depth=9, # profundidade máxima de cada árvore
    subsample=0.31, # taxa de amostragem
    colsample_bytree=0.65, # taxa de amostragem de colunas
    random_state=42 # semente para o gerador de números aleatórios
)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost")
print(f"  MAE  : {mae_xgb:.2f} óbitos")
print(f"  RMSE : {rmse_xgb:.2f} óbitos")
print(f"  R²   : {r2_xgb:.3f}")


## Comparação entre Modelos

In [ ]:
resultados = pd.DataFrame({
    'Modelo': ['Linear Regression', 'Random Forest', 'XGBoost'],
    'MAE': [mae_lr, mae_rf, mae_xgb],
    'RMSE': [rmse_lr, rmse_rf, rmse_xgb],
    'R²': [r2_lr, r2_rf, r2_xgb]
})
print(resultados.sort_values('MAE'))

## Previsões

In [ ]:
# Previsão por REGIÃO
comparacao_regiao = pd.DataFrame({
    'REGIAO': agregado[agregado["MES"] == 12]["REGIAO"].values,
    'Real (óbitos)': y_test.values,
    'Linear': y_pred_lr,
    'RandomForest': y_pred_rf,
    'XGBoost': y_pred_xgb
})

print("\nPrevisões para Dezembro por Região:")
display(comparacao_regiao.groupby('REGIAO').sum().round(1).reset_index())


# Previsão por ESCOLARIDADE
comparacao_escolaridade = pd.DataFrame({
    'ESCOLARIDADE': agregado[agregado["MES"] == 12]["ESCOLARIDADE"].values,
    'Real (óbitos)': y_test.values,
    'Linear': y_pred_lr,
    'RandomForest': y_pred_rf,
    'XGBoost': y_pred_xgb
})

print("\nPrevisões para Dezembro por Escolaridade:")
display(comparacao_escolaridade.groupby('ESCOLARIDADE').sum().round(1).reset_index())

## Importância das Features

In [ ]:
# Importância para Random Forest
rf_importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=True)

plt.figure(figsize=(8,6))
rf_importances.plot(kind='barh', color='skyblue')
plt.title("Importância de Features — Random Forest")
plt.xlabel("Importância")
plt.show()

# Importância para XGBoost
xgb_importances = pd.Series(xgb.feature_importances_, index=features).sort_values(ascending=True)

plt.figure(figsize=(8,6))
xgb_importances.plot(kind='barh', color='salmon')
plt.title("Importância de Features — XGBoost")
plt.xlabel("Importância")
plt.show()


# Coeficientes Regressão Linear
coef_lr = pd.Series(lr.coef_, index=features).sort_values(ascending=True)

plt.figure(figsize=(8,6))
coef_lr.plot(kind='barh', color='violet')
plt.title("Coeficientes do modelo Linear Regression")
plt.xlabel("Valor do coeficiente")
plt.show()
